## Multiclass Venn-ABERS calibration  - Satellite Image dataset

This notebook illustrates the multi-class Venn-ABERS calibration as implemented in:

Manokhin, Valery. "Multi-class probabilistic classification using inductive and cross Venn–Abers predictors." In Conformal and Probabilistic Prediction and Applications, pp. 228-240. PMLR, 2017.

As an example we use the Satellite Image dataset described in more detail here: https://www.openml.org/search?type=data&sort=runs&id=182&status=active

## Import libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff
from sklearn.metrics import log_loss, brier_score_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.calibration import CalibratedClassifierCV

from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

import sys  
sys.path.append('../src/')

import warnings
warnings.filterwarnings('ignore')

# Import Venn-ABERS library
from venn_abers import VennAbersCalibrator


## Load dataset

In [2]:
data = arff.loadarff('dataset_186_satimage.arff')
df = pd.DataFrame(data[0])
df['class'] = ([int(str(i)[2]) for i in df['class'].values])
df['class'] = df['class'].astype('int')-1

In [3]:
df.head()

,Aattr,Battr,Cattr,Dattr,Eattr,Fattr,A1attr,B2attr,C3attr,D4attr,...,D22attr,E23attr,F24attr,A25attr,B26attr,C27attr,D28attr,E29attr,F30attr,class
0,0.117596,1.241362,1.184036,0.815302,-0.158561,1.256483,1.193546,0.818486,-0.141965,0.879481,...,0.807707,-0.069968,1.219160,1.250463,0.597678,-0.054291,1.233342,1.262255,0.603258,0
1,-1.205362,-1.249654,-0.077532,0.444886,-0.895959,-0.447579,-0.786760,-0.554203,-0.364672,0.092157,...,-0.192752,-0.736996,-0.969292,-0.844805,-0.400030,-0.725852,-0.344432,-0.594534,-0.183967,4
2,0.779075,0.148811,0.042617,-0.243030,0.800057,0.164136,0.053370,-0.448612,0.154978,-0.345245,...,-0.877277,0.671174,-0.006373,-0.425752,-0.662584,0.691889,0.356801,-0.175259,-0.236449,6
3,1.146564,0.585831,0.342991,0.021553,0.947536,0.601074,0.353416,0.026550,1.788164,1.010702,...,0.281150,1.412317,1.044084,0.532085,0.282612,1.438068,1.058033,0.842981,0.130923,2
4,-0.764376,-1.162250,-0.137607,0.180303,-0.969698,-1.146681,-0.126658,0.184937,-0.735851,-1.132569,...,-0.192752,-0.885225,-1.231906,-0.784941,-0.347519,-0.875088,-1.220973,-0.774223,-0.551339,4


In [4]:
df['class'].value_counts()

class
0    1531
6    1508
2    1356
4     707
1     703
3     625
Name: count, dtype: int64

There are five classed, with labels 0-6 (class label 5 is missing)

We split the data into a proper training set, calibration set and test set

In [5]:
df_train_cal, df_test = train_test_split(df, test_size = 2000, random_state = 42, shuffle = False)
df_proper_train, df_cal = train_test_split(df_train_cal, test_size = 2000, random_state = 42, shuffle = False)

In [6]:
X_train = df_train_cal.drop('class', axis=1)
y_train = df_train_cal['class']

X_proper_train = df_proper_train.drop('class', axis=1)
y_proper_train = df_proper_train['class']

X_cal = df_cal.drop('class', axis=1)
y_cal = df_cal['class']

X_test = df_test.drop('class', axis=1)
y_test = df_test['class']

In [7]:
lb = LabelBinarizer()
y_test_binary = lb.fit_transform(y_test)

In [8]:
def brier_loss_calc(y_true, prob):
    return ((y_true - prob)**2).mean()

## Classifier comparison

We compare 5 underlying classifiers and two calibration methods (sigmoid and isotonic), both in their inductive and cross-validated format

In [9]:
clfs = {}
clfs['Naive Bayes'] = GaussianNB()
clfs['SVM'] = SVC(probability=True)
clfs['RF'] = RandomForestClassifier()
clfs['XGB'] = AdaBoostClassifier()
clfs['Logistic'] = LogisticRegression(max_iter=10000)
clfs['Neural Network'] =  MLPClassifier(max_iter=10000)


def run_multiclass_comparison(clf_name, clf):

    print(clf_name + ':')
    log_loss_list = []
    brier_loss_list = []
    acc_list = []
    ece_list = []
    
    print('base')
    clf.fit(X_train, y_train)
    p_pred = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('sigmoid')
    clf.fit(X_proper_train, y_proper_train)
    cal_sigm = CalibratedClassifierCV(clf, method='sigmoid', cv='prefit')
    cal_sigm.fit(X_cal, y_cal)
    p_pred = cal_sigm.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('isotonic')
    cal_iso = CalibratedClassifierCV(clf, method='isotonic', cv='prefit')
    cal_iso.fit(X_cal, y_cal)
    p_pred = cal_iso.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('sigmoid_cv')
    cal_sigm_cv = CalibratedClassifierCV(clf, method='sigmoid', cv=5)
    cal_sigm_cv.fit(X_train, y_train)
    p_pred = cal_sigm_cv.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('isotonic_cv')
    cal_iso_cv = CalibratedClassifierCV(clf, method='isotonic', cv=5)
    cal_iso_cv.fit(X_train, y_train)
    p_pred = cal_iso_cv.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred))
    
    print('ivap')
    va = VennAbersCalibrator(clf, inductive=True, cal_size=0.2, random_state=42)
    va.fit(np.asarray(X_train), np.asarray(y_train))
    p_pred_va = va.predict_proba(np.array(X_test))
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred_va))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred_va))
    
    print('cvap')
    va_cv = VennAbersCalibrator(clf, inductive=False, n_splits=5)
    va_cv.fit(np.asarray(X_train), np.asarray(y_train))
    p_pred_cv = va_cv.predict_proba(np.asarray(X_test))
    y_pred = clf.predict(X_test)
    acc_list.append(accuracy_score(y_test, y_pred))
    log_loss_list.append(log_loss(y_test, p_pred_cv))
    brier_loss_list.append(brier_loss_calc(y_test_binary, p_pred_cv))

    print('')
    
    df_ll = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_ll.loc[0] =  [clf_name] + log_loss_list
    df_bl = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_bl.loc[0] =  [clf_name] + brier_loss_list
    df_acc = pd.DataFrame(columns=['Classifier', 'Uncalibrated', 'Platt', 'Isotonic', 'Platt-CV', 'Isotonic-CV', 'IVAP', 'CVAP'])
    df_acc.loc[0] =  [clf_name] + acc_list

    return df_bl, df_ll, df_acc

We compare accuracy, Brier and log loss

In [10]:
results_brier = pd.DataFrame()
results_log = pd.DataFrame()
results_acc = pd.DataFrame()

for clf_name in clfs:
    scratch_b, scratch_l, scratch_acc = run_multiclass_comparison(clf_name, clfs[clf_name])
    results_brier = pd.concat((results_brier, scratch_b), ignore_index=True)
    results_log = pd.concat((results_log, scratch_l), ignore_index=True)
    results_acc = pd.concat((results_acc, scratch_acc), ignore_index=True)
    

Naive Bayes:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

SVM:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

RF:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

XGB:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

Logistic:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap

Neural Network:
base
sigmoid
isotonic
sigmoid_cv
isotonic_cv
ivap
cvap



Calibrating results in a slightly lower accuracy

In [11]:
results_acc.set_index('Classifier', inplace=True)
results_acc.round(3)

,Uncalibrated,Platt,Isotonic,Platt-CV,Isotonic-CV,IVAP,CVAP
Classifier,,,,,,,
Naive Bayes,0.796,0.791,0.791,0.791,0.791,0.791,0.791
SVM,0.894,0.878,0.878,0.878,0.878,0.878,0.878
RF,0.899,0.892,0.892,0.892,0.892,0.892,0.892
XGB,0.750,0.588,0.588,0.588,0.588,0.588,0.588
Logistic,0.862,0.860,0.860,0.860,0.860,0.860,0.860
Neural Network,0.898,0.895,0.895,0.895,0.895,0.895,0.895


However it generally improves Brer and long loss metrics

In [12]:
results_brier.set_index('Classifier', inplace=True)
results_brier.round(4)

,Uncalibrated,Platt,Isotonic,Platt-CV,Isotonic-CV,IVAP,CVAP
Classifier,,,,,,,
Naive Bayes,0.0657,0.0571,0.0480,0.0556,0.0469,0.0449,0.0443
SVM,0.0257,0.0350,0.0309,0.0304,0.0274,0.0273,0.0266
RF,0.0239,0.0254,0.0250,0.0233,0.0230,0.0249,0.0250
XGB,0.0994,0.1025,0.0958,0.0744,0.0613,0.0291,0.0282
Logistic,0.0323,0.0529,0.0481,0.0515,0.0455,0.0317,0.0317
Neural Network,0.0269,0.0302,0.0266,0.0240,0.0224,0.0254,0.0245


In [13]:
results_log.set_index('Classifier', inplace=True)
results_log.round(4)

,Uncalibrated,Platt,Isotonic,Platt-CV,Isotonic-CV,IVAP,CVAP
Classifier,,,,,,,
Naive Bayes,4.0378,0.7704,0.6101,0.7532,0.5960,0.5150,0.5101
SVM,0.2807,0.4341,0.4587,0.3678,0.3335,0.3382,0.3290
RF,0.2676,0.3007,0.3870,0.2744,0.2879,0.2991,0.2991
XGB,1.1760,1.2685,1.2437,0.9034,0.7280,0.3522,0.3418
Logistic,0.3564,0.6104,0.6166,0.5951,0.5857,0.3739,0.3761
Neural Network,0.4088,0.4152,0.4309,0.3177,0.2413,0.3098,0.3000


**Average Brier and log loss ranking**

In [14]:
results_brier.rank(axis=1).mean()

Uncalibrated    4.333333
Platt           6.833333
Isotonic        5.000000
Platt-CV        4.000000
Isotonic-CV     2.666667
IVAP            2.833333
CVAP            2.333333
dtype: float64

In [15]:
results_log.rank(axis=1).mean()

Uncalibrated    3.333333
Platt           6.166667
Isotonic        6.333333
Platt-CV        4.166667
Isotonic-CV     2.833333
IVAP            2.833333
CVAP            2.333333
dtype: float64

In [16]:
results_acc.rank(axis=1).mean()

Uncalibrated    7.0
Platt           3.5
Isotonic        3.5
Platt-CV        3.5
Isotonic-CV     3.5
IVAP            3.5
CVAP            3.5
dtype: float64

**Mean Brier and log loss scores**

In [17]:
results_brier.mean()

Uncalibrated    0.045635
Platt           0.050497
Isotonic        0.045732
Platt-CV        0.043223
Isotonic-CV     0.037722
IVAP            0.030529
CVAP            0.030043
dtype: float64

In [18]:
results_log.mean()

Uncalibrated    1.087881
Platt           0.633208
Isotonic        0.624481
Platt-CV        0.535251
Isotonic-CV     0.462071
IVAP            0.364694
CVAP            0.359356
dtype: float64

Overall, IVAP and CVAP calibrators achive lower Brier and log loss errors